## TOC
<div class="alert alert-block alert-info" style="margin-top: 20px">

1. <a href="#item1">Wrangle wikipedia Postal Code data</a>

2. <a href="#item2">Geocoding</a>

3. <a href="#item3">Group neighbourhoods</a>
    
4. <a href="#item4">Cluster neighbourhoods</a>
    
5. <a href="#item5">Examine Clusters</a>
</div>

## 1. Wrangle wikipedia Postal Code data
<a id='item1'></a>

In [283]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json # library to handle JSON files

#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#import lxml

In [284]:
toronto_boroughs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
toronto_boroughs = toronto_boroughs[0]
toronto_boroughs.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


### Drop rows where Borough is "Not assigned"

In [285]:
toronto_1 = toronto_boroughs[toronto_boroughs['Borough'].map(lambda x: str(x) != "Not assigned")]
toronto_1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Assign name of Borough to Neighbourhoods with value "Not assigned"

In [286]:
toronto_1["Neighbourhood"] = toronto_1["Neighbourhood"].where(cond=toronto_1["Neighbourhood"] != "Not assigned", other=toronto_1["Borough"], axis=0)
toronto_1.head()

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Group Neighbourhoods in comma separated lists, by Borough

In [287]:
toronto_1['Neighbourhood'] = toronto_1[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].\
    transform(lambda x: ','.join(x))

toronto_1 = toronto_1[["Postcode", "Borough", "Neighbourhood"]].drop_duplicates()

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [288]:
# Rename Postcode
toronto_1.columns = ["PostalCode", "Borough", "Neighbourhood"]

In [289]:
# Reset the index
toronto_1.reset_index(drop=True, inplace=True)

In [290]:
# Verify 1 case
toronto_1[toronto_1["PostalCode"] == "M1R"]

,PostalCode,Borough,Neighbourhood
71,M1R,Scarborough,"Maryvale,Wexford"


In [291]:
toronto_1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [292]:
toronto_1.shape

(103, 3)

## 2. Geocoding
<a id='item2'></a>

In [293]:
import geocoder # import geocoder
import json

In [294]:
# Check if this provider works
geojson = geocoder.arcgis('M8Z, Toronto, Ontario').json
geojson

{'address': 'M8Z',
 'bbox': {'northeast': [43.65363000000003, -79.49794976199996],
  'southwest': [43.59563000000002, -79.55594976199995]},
 'confidence': 5,
 'lat': 43.624630000000025,
 'lng': -79.52694976199996,
 'ok': True,
 'quality': 'Postal',
 'raw': {'name': 'M8Z',
  'extent': {'xmin': -79.55594976199995,
   'ymin': 43.59563000000002,
   'xmax': -79.49794976199996,
   'ymax': 43.65363000000003},
  'feature': {'geometry': {'x': -79.52694976199996, 'y': 43.624630000000025},
   'attributes': {'Score': 100, 'Addr_Type': 'Postal'}}},
 'score': 100,
 'status': 'OK'}

In [295]:
# The google provider doesn't work, this one seems to do (and it doesn't need the silly while loop)
def getLatLng(postal_code):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code)).json
    lat = g["lat"]
    lng = g["lng"]
    return lat, lng

In [296]:
# Check if the function works
getLatLng("M8Z")

(43.624630000000025, -79.52694976199996)

In [297]:
# Apply the function to the entire dataframe
toronto_1["lat", "lng"] = toronto_1.apply(lambda x: getLatLng(x.PostalCode), axis=1 )

In [298]:
toronto_1.head()

,PostalCode,Borough,Neighbourhood,"(lat, lng)"
0,M3A,North York,Parkwoods,"(43.75242000000003, -79.32924245299995)"
1,M4A,North York,Victoria Village,"(43.73060024600005, -79.31326499999994)"
2,M5A,Downtown Toronto,Harbourfront,"(43.65029500000003, -79.35916572299999)"
3,M6A,North York,"Lawrence Heights,Lawrence Manor","(43.72327000000007, -79.45128601699997)"
4,M7A,Queen's Park,Queen's Park,"(43.66115033500006, -79.39171499999998)"


In [300]:
# Create 2 columns for lat and lng instead of one column containing tuples
toronto_1[["lat", "lng"]] = pd.DataFrame( toronto_1[("lat", "lng")].tolist() )
toronto_1.head()

,PostalCode,Borough,Neighbourhood,"(lat, lng)",lat,lng
0,M3A,North York,Parkwoods,"(43.75242000000003, -79.32924245299995)",43.752420,-79.329242
1,M4A,North York,Victoria Village,"(43.73060024600005, -79.31326499999994)",43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,"(43.65029500000003, -79.35916572299999)",43.650295,-79.359166
3,M6A,North York,"Lawrence Heights,Lawrence Manor","(43.72327000000007, -79.45128601699997)",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,"(43.66115033500006, -79.39171499999998)",43.661150,-79.391715


In [301]:
# Drop the old tuple column
toronto_1.drop([("lat", "lng")], axis=1, inplace=True)

In [302]:
toronto_1.head()

,PostalCode,Borough,Neighbourhood,lat,lng
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715


## 3. Group neighbourhoods
<a id='item3'></a>

In [303]:
CLIENT_ID = 'O3R0SYR4X4XVKABB0Z5L2EDCNNX2XHCAA5BWAXK5S4QJSZ5P' # your Foursquare ID
CLIENT_SECRET = '3SYACT4SXOICXJNGT3G2OAF1G5LLK1X1BKO5VVZB2OI2BWX3' # your Foursquare Secret
VERSION = '20191201'



print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O3R0SYR4X4XVKABB0Z5L2EDCNNX2XHCAA5BWAXK5S4QJSZ5P
CLIENT_SECRET:3SYACT4SXOICXJNGT3G2OAF1G5LLK1X1BKO5VVZB2OI2BWX3


In [304]:
toronto_1

,PostalCode,Borough,Neighbourhood,lat,lng
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715
5,M9A,Downtown Toronto,Queen's Park,43.662299,-79.528195
6,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749055,-79.362227
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


In [309]:
limit=100
radius=500
lat = toronto_1.loc[100, "lat"]
lng = toronto_1.loc[100, "lng"]

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=O3R0SYR4X4XVKABB0Z5L2EDCNNX2XHCAA5BWAXK5S4QJSZ5P&client_secret=3SYACT4SXOICXJNGT3G2OAF1G5LLK1X1BKO5VVZB2OI2BWX3&ll=43.648690000000045,-79.38543999999996&v=20191201&radius=500&limit=100'

In [310]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df522d2153fd432b28dc065'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Entertainment District',
  'headerFullLocation': 'Entertainment District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 148,
  'suggestedBounds': {'ne': {'lat': 43.65319000450005,
    'lng': -79.37923257586083},
   'sw': {'lat': 43.64418999550004, 'lng': -79.3916474241391}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50322b6ae4b09116a296568c',
       'name': 'Soho House Toronto',
       'location': {'address': '192 Adelaide St W',
        'crossStreet': 'at Simcoe St.',
        'lat': 43.64873430659662,
        'lng': -79.38654117564838,

In [311]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [313]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Soho House Toronto,Speakeasy,43.648734,-79.386541
1,Shangri-La Toronto,Hotel,43.649129,-79.386557
2,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
3,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
4,Pizzeria Libretto,Pizza Place,43.648334,-79.385111


In [314]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [317]:
# Function to collect information about the venues in each neighbourhood
# Iterate over passed lists of neighbourhoods, latitude and longitude;
# 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [322]:
# type your answer here
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto_1['Neighbourhood'],
                                   latitudes=toronto_1['lat'],
                                   longitudes=toronto_1['lng']
                                  )



Parkwoods
Victoria Village
Harbourfront
Lawrence Heights,Lawrence Manor
Queen's Park
Queen's Park
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design Exchange,Toronto 

In [324]:
print(toronto_venues.shape)
toronto_venues.head()

(2467, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73060,-79.313265,Stellar's Lenonade Stand,43.731507,-79.311712,Food Stand
3,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73060,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store


In [327]:
toronto_venues.groupby(["Neighborhood"]).count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,13,13,13,13,13,13
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",1,1,1,1,1,1
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",14,14,14,14,14,14
"Alderwood,Long Branch",5,5,5,5,5,5


In [343]:
toronto_venues.groupby(["Venue Category"]).count()["Neighborhood"].sort_values(ascending=False)

Venue Category
Coffee Shop                                 214
Café                                        102
Restaurant                                   71
Bakery                                       59
Pizza Place                                  57
Hotel                                        57
Italian Restaurant                           52
Park                                         52
Bar                                          49
Clothing Store                               46
Sandwich Place                               42
Fast Food Restaurant                         40
Japanese Restaurant                          38
Gym                                          36
American Restaurant                          35
Sushi Restaurant                             35
Thai Restaurant                              32
Pub                                          32
Grocery Store                                32
Steakhouse                                   31
Gastropub                

,Neighborhood
Venue Category,
Coffee Shop,214
Café,102
Restaurant,71
Bakery,59
Pizza Place,57
Hotel,57
Italian Restaurant,52
Park,52
Bar,49


In [345]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [346]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,0

In [347]:
toronto_grouped.shape

(100, 268)

#### Let's print each neighborhood along with the top 5 most common venues

In [348]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2                Hotel  0.05
3           Steakhouse  0.04
4  American Restaurant  0.03


----Agincourt----
                  venue  freq
0         Shopping Mall  0.15
1  Hong Kong Restaurant  0.08
2                  Park  0.08
3                Bakery  0.08
4       Badminton Court  0.08


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0           Pharmacy   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3             Museum   0.0
4      Moving Target   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.14
1   Fried Chicken Joint  0.07
2  Fast Food Restaurant  0.07
3        Sandwich Place  0.07
4              Pharmacy  0.07


----Alderwood,Long Branch----
               venue  freq
0  Convenience St

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [349]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [371]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gastropub,Bakery,Asian Restaurant,Japanese Restaurant,Bar,American Restaurant
1,Agincourt,Shopping Mall,Park,Hong Kong Restaurant,Bakery,Supermarket,Badminton Court,Chinese Restaurant,Department Store,Pool,Sushi Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Women's Store,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Sandwich Place,Pharmacy,Park,Hardware Store,Video Store,Pizza Place,Fast Food Restaurant,Beer Store
4,"Alderwood,Long Branch",Pub,Sandwich Place,Convenience Store,Gym,Dance Studio,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


## 4. Cluster neighbourhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [443]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [444]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 6, 6, 0, 6, 2, 1, 6, 6, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [445]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)

In [446]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [447]:
toronto_merged = toronto_1
neighborhoods_venues_sorted.dropna()
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gastropub,Bakery,Asian Restaurant,Japanese Restaurant,Bar,American Restaurant
1,6,Agincourt,Shopping Mall,Park,Hong Kong Restaurant,Bakery,Supermarket,Badminton Court,Chinese Restaurant,Department Store,Pool,Sushi Restaurant
2,6,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Women's Store,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
3,0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Sandwich Place,Pharmacy,Park,Hardware Store,Video Store,Pizza Place,Fast Food Restaurant,Beer Store
4,6,"Alderwood,Long Branch",Pub,Sandwich Place,Convenience Store,Gym,Dance Studio,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
5,2,"Bathurst Manor,Downsview North,Wilson Heights",Men's Store,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
6,1,Bayview Village,Golf Driving Range,Construction & Landscaping,Park,Dog Run,Trail,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
7,6,"Bedford Park,Lawrence Manor East",Italian Restaurant,Sushi Restaurant,Coffee Shop,Sandwich Place,Restaurant,Fast Food Restaurant,Indian Restaurant,Juice Bar,Sports Club,Café
8,6,Berczy Park,Coffee Shop,Restaurant,Café,Seafood Restaurant,Cheese Shop,Bakery,Cocktail Bar,Farmers Market,Hotel,Beer Bar
9,1,"Birch Cliff,Cliffside West",College Stadium,Gym Pool,General Entertainment,Gym,Skating Rink,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant


In [448]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.dropna()
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,1.0,Food & Drink Shop,Park,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.730600,-79.313265,1.0,Grocery Store,Food Stand,Park,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,6.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Pet Store,Breakfast Spot,Brewery,Shoe Store,Café,Pub
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286,6.0,Clothing Store,American Restaurant,Furniture / Home Store,Men's Store,Toy / Game Store,Cosmetics Shop,Burger Joint,Fast Food Restaurant,Food Court,Mexican Restaurant
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715,6.0,Coffee Shop,Sandwich Place,Café,Pharmacy,Theater,Italian Restaurant,Mediterranean Restaurant,Park,Salad Place,Smoothie Shop


In [449]:
toronto_merged = toronto_merged[toronto_merged["Cluster Labels"] >= 0]
toronto_merged.shape

(101, 16)

In [450]:
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

In [451]:
toronto_merged.head()

,PostalCode,Borough,Neighbourhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,1,Food & Drink Shop,Park,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.730600,-79.313265,1,Grocery Store,Food Stand,Park,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,6,Coffee Shop,Bakery,Boat or Ferry,Theater,Pet Store,Breakfast Spot,Brewery,Shoe Store,Café,Pub
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286,6,Clothing Store,American Restaurant,Furniture / Home Store,Men's Store,Toy / Game Store,Cosmetics Shop,Burger Joint,Fast Food Restaurant,Food Court,Mexican Restaurant
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715,6,Coffee Shop,Sandwich Place,Café,Pharmacy,Theater,Italian Restaurant,Mediterranean Restaurant,Park,Salad Place,Smoothie Shop


In [452]:
# Toronto centre
latitude = 43.73
longitude = -79.3

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [453]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [455]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lng'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [441]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Canadian")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

<a id='item5'></a>

## 5. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1
The boroughs contained in this cluster appear to have an abundance of Fast Foods, Convenience stores and Pizza places. Seems like they could be rather cheap places to live.

In [456]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Bus Line,Breakfast Spot,Café,Rock Climbing Spot,Bank,Gastropub
10,North York,0,Pizza Place,Fast Food Restaurant,Spa,Gas Station,Sushi Restaurant,Asian Restaurant,Latin American Restaurant,Japanese Restaurant,Grocery Store,Mediterranean Restaurant
13,North York,0,Bubble Tea Shop,Coffee Shop,Beer Store,Intersection,Supermarket,Gym,Grocery Store,Electronics Store,Ethiopian Restaurant,Event Space
14,East York,0,Bus Line,Pharmacy,Grocery Store,Pizza Place,Coffee Shop,Market,Breakfast Spot,Café,Metro Station,Middle Eastern Restaurant
16,York,0,Trail,Hockey Arena,Field,Park,Grocery Store,Coffee Shop,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
17,Etobicoke,0,Carpet Store,College Rec Center,Electronics Store,Shopping Mall,Grocery Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Filipino Restaurant
32,Scarborough,0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Filipino Restaurant,Dog Run
38,Scarborough,0,Discount Store,Hobby Shop,Convenience Store,Chinese Restaurant,Department Store,Coffee Shop,Farm,Electronics Store,Ethiopian Restaurant,Event Space
41,East Toronto,0,Discount Store,Restaurant,Park,Bus Line,Grocery Store,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
46,North York,0,Convenience Store,Hotel,Spa,Fast Food Restaurant,Electronics Store,Beer Store,Bank,Clothing Store,Coffee Shop,Bakery


#### Cluster 2
The boroughs contained in this cluster contain Parks, Piers, Gardens, and Gas Stations. Probably rather far from the centre but with some green areas.

In [458]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Food & Drink Shop,Park,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
1,North York,1,Grocery Store,Food Stand,Park,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
7,North York,1,Gas Station,Burger Joint,Park,Women's Store,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
18,Scarborough,1,Construction & Landscaping,Gym / Fitness Center,Park,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
21,York,1,Park,Women's Store,Bakery,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Beer Store,Market,Gym,Cuban Restaurant
36,Downtown Toronto,1,Pier,Harbor / Marina,Park,Women's Store,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
39,North York,1,Golf Driving Range,Construction & Landscaping,Park,Dog Run,Trail,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
49,North York,1,Bakery,Park,Farmers Market,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Women's Store
58,Scarborough,1,College Stadium,Gym Pool,General Entertainment,Gym,Skating Rink,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant
66,North York,1,Speakeasy,Bank,Convenience Store,Park,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


#### Cluster 3
A tiny cluster, the only Borough in this cluster contains Men's and Women's stores, plus a few restaurants. Probably a mall.

In [459]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,2,Men's Store,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 4
A tiny cluster, the only Borough in this cluster contains Women's stores, plus a few restaurants. Probably a mall.

In [460]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Pet Store,Women's Store,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


#### Cluster 5
This tiny cluster contains many auto garages, probably not the ideal place to live (at least for me).

In [461]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,4,Auto Garage,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Donut Shop


#### Cluster 6
The boroughs contained in this cluster contain Business Services, Food courts and coffees. Seems like a central place with lots of work opportunities.

In [462]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Scarborough,5,Coffee Shop,Park,Business Service,Korean Restaurant,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Women's Store
40,North York,5,Food Court,Airport,Park,Coffee Shop,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
53,North York,5,Business Service,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
57,North York,5,Coffee Shop,Nightclub,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Women's Store,Donut Shop
